## DQN baselines LSTM + MLP

In [1]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNLSTMMLPBaselineAgent
from tqdm.auto import tqdm
import random
import itertools

# default
room_size = "xl-different-prob"
target_update_interval = 10
history_block_size = 16
batch_size = 16
terminates_at = 99
num_iterations = (terminates_at + 1) * 100
validation_starts_at = num_iterations // 2
gamma = 0.9

prob_type = (
    "non-equal-object-probs" if "different-prob" in room_size else "equal-object-probs"
)
root_path = f"./training-results/{prob_type}/baselines/room_size={room_size}/history_block_size={history_block_size}/"


# Number of combinations you want
num_combinations = 100  # Change this to however many combinations you need

# random
test_seed_ = [i for i in range(num_combinations)]
replay_buffer_size_ = [
    num_iterations,
    # num_iterations // 2,
]
warm_start_ = [
    # num_iterations // 2,
    num_iterations // 5,
    # num_iterations // 10,
]

# Generate all combinations
params_all = list(
    itertools.product(
        test_seed_,
        replay_buffer_size_,
        warm_start_,
    )
)

# Get random combinations
random_combinations = random.sample(params_all, num_combinations)

for test_seed, replay_buffer_size, warm_start in tqdm(random_combinations):
    params = {
        "env_str": "room_env:RoomEnv-v2",
        "num_iterations": num_iterations,
        "replay_buffer_size": replay_buffer_size,
        "validation_starts_at": validation_starts_at,
        "warm_start": warm_start,
        "batch_size": batch_size,
        "target_update_interval": target_update_interval,
        "epsilon_decay_until": num_iterations,
        "max_epsilon": 1.0,
        "min_epsilon": 0.1,
        "gamma": gamma,
        "history_block_size": history_block_size,
        "lstm_params": {
            "hidden_size": 96,
            "num_layers": 2,
            "embedding_dim": 96,
            "bidirectional": False,
        },
        "mlp_params": {
            "hidden_size": 96,
            "num_hidden_layers": 1,
            "dueling_dqn": True,
        },
        "num_samples_for_results": {"val": 5, "test": 10},
        "validation_interval": 5,
        "plotting_interval": 50,
        "train_seed": test_seed + 5,
        "test_seed": test_seed,
        "device": "cpu",
        "env_config": {
            "question_prob": 1.0,
            "terminates_at": terminates_at,
            "randomize_observations": "objects",
            "room_size": room_size,
            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
            "make_everything_static": False,
            "num_total_questions": 1000,
            "question_interval": 1,
            "include_walls_in_observations": True,
            "deterministic_objects": False,
        },
        "default_root_dir": root_path,
        "run_handcrafted_baselines": True,
    }

    agent = DQNLSTMMLPBaselineAgent(**params)
    agent.train()

/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/room_env/envs/room2.py:990: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `reset(

KeyboardInterrupt: 